In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import power_transform

In [2]:
df=pd.read_csv("Data2/ufc-master.csv")

df_upcoming=pd.read_csv("Data2/upcoming-event.csv")

df_recent=pd.read_csv("Data2/most-recent-event.csv")

upcoming_fights=len(df_upcoming)
print(upcoming_fights)

11


In [3]:
df_update=df_recent.append(df)

In [4]:
df_combined=df_upcoming.append(df_update)

In [5]:
df_combined['label']=''


mask = df_combined['Winner'] == 'Red'
df_combined['label'][mask] = 0
mask = df_combined['Winner'] == 'Blue'
df_combined['label'][mask] = 1


df_combined['label'] = pd.to_numeric(df_combined['label'], errors='coerce')


df_combined['date'] = pd.to_datetime(df['date'])

C:\Users\nateb\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\nateb\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [6]:
label_df = df_combined['label']

df_train = df_combined[upcoming_fights:]
label_train = label_df[upcoming_fights:]

df_test = df_combined[:upcoming_fights]
label_test = label_df[:upcoming_fights]


#Make sure the sizes are the same
print(len(df_test))
print(len(label_test))

print(len(df_train))
print(len(label_train))

11
11
4434
4434


In [7]:
from sklearn.tree import DecisionTreeClassifier
my_model=DecisionTreeClassifier(max_depth=5)

In [8]:
my_features = ['R_odds', 'B_Stance']

#Let's grab the names of the fighters for the upcoming event
#This will be useful to print predictions at the end.
fighters_test = df_test[['R_fighter', 'B_fighter']]
odds_test = df_test[['R_odds', 'B_odds']]

In [9]:
df_train_prepped = df_train[my_features].copy()
df_test_prepped = df_test[my_features].copy()

In [10]:

df_train_prepped = pd.get_dummies(df_train_prepped)
df_test_prepped = pd.get_dummies(df_test_prepped)

#Ensure both sets are dummified the same
df_train_prepped, df_test_prepped = df_train_prepped.align(df_test_prepped, join='left', axis=1) 

df_test_prepped = df_test_prepped.fillna(0)

#Since we may have dropped some rows we need to drop the matching rows in the labels
label_train_prepped = label_train[label_train.index.isin(df_train_prepped.index)]
label_test_prepped = label_test[label_test.index.isin(df_test_prepped.index)]
fighters_test_prepped = fighters_test[fighters_test.index.isin(df_test_prepped.index)]
odds_test_prepped = odds_test[odds_test.index.isin(df_test_prepped.index)]

In [11]:
#Quick test that lengths match.
print(len(label_train_prepped))
print(len(df_train_prepped))
print(len(label_test_prepped))
print(len(df_test_prepped))
print(len(fighters_test_prepped))
print(len(odds_test_prepped))

4434
4434
11
11
11
11


In [12]:
my_model.fit(df_train_prepped, label_train_prepped)

probs = my_model.predict_proba(df_test_prepped)

probs

array([[0.42613636, 0.57386364],
       [0.42613636, 0.57386364],
       [0.8297456 , 0.1702544 ],
       [0.42613636, 0.57386364],
       [0.54440789, 0.45559211],
       [0.8297456 , 0.1702544 ],
       [0.6819788 , 0.3180212 ],
       [0.71984436, 0.28015564],
       [0.54440789, 0.45559211],
       [0.33828996, 0.66171004],
       [0.75806452, 0.24193548]])

In [13]:
fighter_array=fighters_test_prepped.to_numpy()
odds_array=odds_test_prepped.to_numpy()

In [14]:
probs_list=np.array(list(zip(fighter_array, odds_array, probs)))

probs_list

array([[['Brian Ortega', 'Chan Sung Jung'],
        [160, -200],
        [0.42613636363636365, 0.5738636363636364]],

       [['Katlyn Chookagian', 'Jessica Andrade'],
        [130, -159],
        [0.42613636363636365, 0.5738636363636364]],

       [['Jimmy Crute', 'Modestas Bukauskas'],
        [-335, 250],
        [0.8297455968688845, 0.17025440313111545]],

       [['Claudio Silva', 'James Krause'],
        [130, -162],
        [0.42613636363636365, 0.5738636363636364]],

       [['Thomas Almeida', 'Jonathan Martinez'],
        [-139, 110],
        [0.5444078947368421, 0.4555921052631579]],

       [['Mateusz Gamrot', 'Guram Kutateladze'],
        [-335, 245],
        [0.8297455968688845, 0.17025440313111545]],

       [['Gillian Robertson', 'Poliana Botelho'],
        [-215, 169],
        [0.6819787985865724, 0.31802120141342755]],

       [['Junyong Park', 'John Phillips'],
        [-278, 215],
        [0.7198443579766537, 0.2801556420233463]],

       [['Jamie Mullarkey', 'Fares 

In [15]:
def get_bet_ev(odds, probs):
    if odds>0:
        return ((odds*probs)-(100*(1-probs)))
    else:
        return ((100/abs(odds))*100*probs-(100*(1-probs)))


In [16]:
for p in probs_list:
    red_ev=get_bet_ev(p[1][0], p[2][0])
    blue_ev=get_bet_ev(p[1][1], p[2][1])
    
    print(p[0][0], "(RED) vs ", p[0][1], "(BLUE)")
    print(p[0][0], "has a", "%.2f" % (p[2][0]*100), "percent chance of winning.  Their odds are", p[1][0], "This gives them a single bet EV of", "%.2f" %red_ev)
    print(p[0][1], "has a", "%.2f" % (p[2][1]*100), "percent chance of winning.  Their odds are", p[1][1], "This gives them a single bet EV of", "%.2f" %blue_ev)
    if red_ev > 0:
        print("RED is a good bet")
    elif (blue_ev > 0):
        print("BLUE is a good bet")
    else:
        print("There is NO good bet")
    
    print()

Brian Ortega (RED) vs  Chan Sung Jung (BLUE)
Brian Ortega has a 42.61 percent chance of winning.  Their odds are 160 This gives them a single bet EV of 10.80
Chan Sung Jung has a 57.39 percent chance of winning.  Their odds are -200 This gives them a single bet EV of -13.92
RED is a good bet

Katlyn Chookagian (RED) vs  Jessica Andrade (BLUE)
Katlyn Chookagian has a 42.61 percent chance of winning.  Their odds are 130 This gives them a single bet EV of -1.99
Jessica Andrade has a 57.39 percent chance of winning.  Their odds are -159 This gives them a single bet EV of -6.52
There is NO good bet

Jimmy Crute (RED) vs  Modestas Bukauskas (BLUE)
Jimmy Crute has a 82.97 percent chance of winning.  Their odds are -335 This gives them a single bet EV of 7.74
Modestas Bukauskas has a 17.03 percent chance of winning.  Their odds are 250 This gives them a single bet EV of -40.41
RED is a good bet

Claudio Silva (RED) vs  James Krause (BLUE)
Claudio Silva has a 42.61 percent chance of winning.  T